In [ ]:
from temporal_grounding.temporal_grounding import extract_timestamps_from_video

vp = r"testing\shot_retrevial\videos\940432_2900_KK MZT Skopje Aerodrom_2953_KK Buducnost_period2.avi"
sp = r"testing\shot_retrevial\timestamps\940432_2900_KK MZT Skopje Aerodrom_2953_KK Buducnost_period2.json"
extract_timestamps_from_video(vp, sp)

In [ ]:
from temporal_grounding.temporal_grounding import convert_float_to_time_str, convert_time_to_float
from data_prep import *
from datascience import *
import numpy as np

log_path = get_log_path("972173")
log_path

start_time = 600
# convert_float_to_time_str(531.199)
convert_time_to_float("5:13")

# arr = []
# with open(log_path) as file:
#     doc = csv.reader(file, delimiter=';')
#     for row in doc:
#         arr.append(row)

# for row in arr[1: ]:
#     if row[2][0] == "2" or row[2][0] == "3":
#         print("Shot", row[26], "at", row[23])


In [ ]:
import cv2

def clip_video(input_path, output_path, start_frame, end_frame):
    try:
        cap = cv2.VideoCapture(input_path)
        if not cap.isOpened():
            raise Exception("Error: Could not open video file")
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            out.write(frame)
            if cap.get(cv2.CAP_PROP_POS_FRAMES) == end_frame:
                break
        cap.release()
        out.release()
    except Exception as e:
        print(f"Error: {str(e)}")

In [ ]:
from data_prep import *
import os
import json

video_path = r"testing\shot_retrevial\videos\940432_2900_KK MZT Skopje Aerodrom_2953_KK Buducnost_period2.avi"
shots_out_path = r"testing\shot_retrevial\shot_clips"
timestamps_path = r"testing\shot_retrevial\timestamps\940432_2900_KK MZT Skopje Aerodrom_2953_KK Buducnost_period2.json"

def extract_shots_from_video(video_path, shots_out_dir, timestamps_path=None):

    try:
        game_id = os.path.basename(video_path).split("_")[0]
    except:
        raise Exception(f"Bad video path: {video_path}")
    try:
        with open(timestamps_path, "r") as f:
            timestamps = json.load(f)
    except:
        raise Exception(f"Error: bad timestamps path: {timestamps_path}.")

    path_to_logs = get_log_path(game_id)
    shot_events = get_shot_events(path_to_logs)
    intervals = []
    for event in shot_events:
        key = f"{event[2]}_{event[3]}"
        if key in timestamps:
            try:
                backwards_pad = 15
                shot_duration = 30 * 3
                start_frame = timestamps[key] - backwards_pad
                end_frame = start_frame + shot_duration
                intervals.append([start_frame, end_frame, event])
            except:
                pass

    shot_index = 0
    video_title = os.path.basename(video_path)
    shot_subdir = os.path.join(shots_out_dir, video_title.split(".")[0])
    os.makedirs(shot_subdir, exist_ok=True)
    for interval in intervals:
        start_frame, end_frame = interval[0], interval[1]
        is_shot_made = interval[2][1] 
        output_path = os.path.join(shot_subdir, f"_clip_{shot_index}_{is_shot_made}_.avi")
        print(f"Shot clips saved to: {output_path}")
        clip_video(video_path, output_path, start_frame, end_frame)
        shot_index += 1
        

extract_shots_from_video(video_path, shots_out_path, timestamps_path)

In [ ]:
( 6 * 60 ) + 30

# Gamelog Imports & Formatting
***

In [12]:
from datascience import *
import csv
import pandas as pd
from data_prep import get_log_path
from temporal_grounding.temporal_grounding import convert_float_to_time_str
import numpy as np

id = "936399"
path = get_log_path(id)
df = pd.read_csv(path, delimiter=';', skiprows=1)


In [39]:
def build_table_from_game_log(game_id, quarter) -> Table:

    path = get_log_path(game_id)
    if path == '':
        print(f"No log available for game with id {id}")
        return Table()
    df = pd.read_csv(path, delimiter=';', skiprows=1)

    t = Table.from_df(df)
    columns = [str(i) for i in range(31)]
    labels = np.array(t.labels)
    t = t.relabel(labels, columns).select("2", "13", "22", "23")
    t = t.relabel("2", "event")
    t = t.relabel("22", "time_elapsed")
    t = t.relabel("13", "quarter")
    t = t.relabel("23", "time_elapsed_high")
    start_time = int(max(t.column("time_elapsed_high")))
    time_elapsed_col = t.column("time_elapsed")
    time_remaining_col = []
    for val in time_elapsed_col:
        time = start_time-val
        time_remaining_col.append(
            convert_float_to_time_str(time))
        
    time_elapsed_high_col = t.column("time_elapsed_high")
    time_remaining_col_high = []
    for val in time_elapsed_high_col:
        time = start_time-val
        time_remaining_col_high.append(
            convert_float_to_time_str(time))

    t = t.with_columns("time_remaining", time_remaining_col, 
                       "time_remaining_high", time_remaining_col_high
                       )
    # t = t.where("event", (are.containing("2") | are.containing("3"))
    #             & are.not_containing("F"))
    t = t.where("quarter", are.equal_to(quarter))
    t = t.move_column("time_remaining", 2)
    t = t.move_column("time_remaining_high", 3)
    return t


In [45]:
import numpy as np

id = "971400"
build_table_from_game_log(id, 4).show(150)


event,quarter,time_remaining,time_remaining_high,time_elapsed,time_elapsed_high
2-,4,10:00.0,09:50.0,0,10
Ball in play,4,10:00.0,09:50.0,0,10
Accurate pass,4,09:48.0,09:39.0,12,21
Assisting,4,09:48.0,09:39.0,12,21
2+,4,09:48.0,09:39.0,12,21
Ball in play,4,09:48.0,09:39.0,12,21
2+,4,09:39.0,09:12.0,21,48
Assisting,4,09:39.0,09:12.0,21,48
Pick'n'Roll,4,09:39.0,09:12.0,21,48
Pick'n'Roll,4,09:12.0,08:54.0,48,66


In [43]:
t = Table.from_df(df)
columns = [str(i) for i in range(31)]
labels = np.array(t.labels)
t = t.relabel(labels, columns).select("2", "13", "22", "23")
t = t.relabel("2", "event")
t = t.relabel("22", "time_elapsed")
t = t.relabel("13", "quarter")
t = t.relabel("23", "time_elapsed_high")
start_time = int(max(t.column("time_elapsed_high")))
time_elapsed_col = t.column("time_elapsed")
time_remaining_col = []
for val in time_elapsed_col:
    time = start_time-val
    time_remaining_col.append(
        convert_float_to_time_str(time))
t = t.with_column("time_remaining", time_remaining_col)
t = t.where("event", (are.containing("2") | are.containing("3"))
            & are.not_containing("F"))
t

event,quarter,time_elapsed,time_elapsed_high,time_remaining
2-,1,22.38,39.944,09:37.6
3+,1,50.436,56.67,09:09.5
2+,1,61.134,78.85,08:58.8
2-,1,81.809,93.614,08:38.1
3-,1,95.134,100.375,08:24.8
2+,1,102.754,116.381,08:17.2
2-,1,131.48,137.649,07:48.5
3-,1,150.106,150.836,07:29.8
2+,1,153.406,164.006,07:26.5
2+,1,169.446,189.787,07:10.5


# Shot-Extraction w/ Logs Only

In [47]:
from data_prep import *
from shot_clip_extraction import *
import cv2

def get_shot_events_mod(csv_path: str, period: int):
    """
    Given a path to a game csv, return all moments at which a shot occured.
    Moment: [quarter, time_remaining].

    Discards free-throws.
    """

    shots = []

    try:
        with open(csv_path) as file:
            doc = csv.reader(file, delimiter=';')
            for row in doc:
                event = row[2]
                quarter = row[13]
                event_timestamp = row[14]
                ev_chr_1 = row[2][0]
                event_condition = ("2" == ev_chr_1 or "3" == ev_chr_1) and ("F" not in event) and (quarter == period)
                if event_condition:
                    shots.append({
                        "event": event,
                        "timestamp": float(event_timestamp)
                    })
    except:
        print(f"Error: could not open csv for path: {csv_path}")
        return []
    
    return shots

def extract_shots_from_video_mod(path_to_video, clips_dir):

    try:
        game_id = os.path.basename(path_to_video).split("_")[0]
        period = path_to_video[-5]
    except:
        raise Exception(f"Bad video path: {path_to_video}")
    path_to_logs = get_log_path(game_id)
    if path_to_logs == "":
        print(f"Error: no game logs found for video at {path_to_video}")
        return
    
    cap = cv2.VideoCapture(path_to_video)
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    shot_events = get_shot_events_mod(path_to_logs, period)
    for shot in shot_events:
        event = shot["event"]
        timestamp = float(shot["timestamp"])
        framestamp = int(fps * timestamp)
        start_frame = int(framestamp - (fps * 2))
        end_frame = int(framestamp + (fps * 2))
        out_path = os.path.join(clips_dir, f"{event}_{game_id}_{timestamp}.mp4")
        clip_video(path_to_video, out_path, start_frame, end_frame)
        
        

In [48]:
vid_path = r"D:\400\936097_10535_Ratiopharm Ulm_2918_BC Lietkabelis Panevezys_period1.mp4"
shots_dir = r"D:\shot-clips-labeled"

extract_shots_from_video_mod(vid_path, shots_dir)


In [36]:
generate_game_log_map("data\gamelogs_map.json", r"E:\Desktop\hudl-logs\DATA\game_logs")